In [1]:
%run _BaseInfo.ipynb

In [2]:
import time

In [3]:
from openpyxl import load_workbook
def save_excel(file_name,df):
    sheet_name='Sheet1'

    try:
        # 嘗試加載現有的 Excel 文件
        book = load_workbook(file_name)

        with pd.ExcelWriter(file_name, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
            # 加載現有數據
            writer.book = book
            if sheet_name in book.sheetnames:
                existing_df = pd.read_excel(file_name, sheet_name=sheet_name)
                df = pd.concat([existing_df, df], ignore_index=True)

            # 將 DataFrame 寫入指定的 Sheet
            df.to_excel(writer, sheet_name=sheet_name, index=False)

            writer.save()

    except FileNotFoundError:
        # 如果文件不存在，創建新文件並寫入 DataFrame
        with pd.ExcelWriter(file_name, engine='openpyxl') as writer:
            df.to_excel(writer, sheet_name=sheet_name, index=False)

In [4]:
code='1101'
for code in _baseInfo.dropna()['公司代號'].iloc[::-1]:
    try:
        time.sleep(3)
        url='https://pscnetinvest.moneydj.com/z/zc/zca/zca.djhtm?a='+code
        response = requests.get(url,verify = False)
        response.encoding = 'big5'
        df = pd.DataFrame()
        df = pd.concat([df, pd.DataFrame([[str(code)]], columns=['stock_number'])], ignore_index=True)


        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            table=soup.find('table').find('table').find('table')

            for x in range(len(table.find_all('tr'))):
                #if x>=1 and x<5:
                if x!=0 and x!=5 and (x<12 or x==20) :
                    detail=[th.text.replace('', '') for th in table.find_all('tr')[x].find_all('td')]
                    for y in range(len(detail)-1):

                        if (y % 2) == 0:
                            df[detail[y]]=str(detail[y+1])
            #df.columns       
            save_df=df[['stock_number', '收盤價' ,'每股淨值(元)','股價淨值比','本益比', '一年內最高價', '一年內最低價', '一年內最大量', '一年內最低量','營收比重']]
            file_name='Result/StockInfo.xlsx'
            save_excel(file_name,save_df)
    except:
            print('error'+code)